<h1 style="text-align: center; font-size: 500%; text-decoration: underline;">Concluding Assignment<h1>
<h2 style="text-align: center; font-size: 250%;">Ido Israeli (ID - 212432439)<br>Jonathan Derhy (ID - 315856377)<h2>
<h2 style="text-align: center; font-size: 400%; text-decoration: underline;">Part II<h2>

<h4 style="text-decoration: underline;">Imports<h4>

In [134]:
import sys
import string
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

<h5 style="text-decoration: underline;">Functions for Debugging</h5>

In [135]:
#
def print_variable_data(variable, name=None):
    if not isinstance(name, str):
        name = "Unnamed Variable"
    print(f'-------------------{name}-------------------\n')
    print(f'Type: {type(variable)}')
    if isinstance(variable, np.ndarray): print(f'Shape: {variable.shape}')
    elif isinstance(variable, list): print(f'Shape: {len(variable)}')
    print(f'contents:\n{variable}')
    print("\n--------------------------------------\n")

#
def validate_instance(variable, type):
    if not isinstance(variable, type):
        print(f'ERROR!\nVariable must be an instance of {type}!')
        sys.exit(1)


<h3 style="text-decoration: underline;">Part A: Preparing the data to train a linear classifier<h3>

<h5>Functions for Selecting Train and Test Data</h5>

In [136]:
#
def shuffle_dataframe(df):
    validate_instance(df, pd.DataFrame)
    return df.sample(frac=1)

#
def select_train_and_test_by_train_percentage(df, train_percentage, toShuffle=False):
    validate_instance(df, pd.DataFrame)
    if toShuffle:
        train = df.sample(frac=train_percentage)
        test = df.drop(train.index)
    elif ~toShuffle:
        num_of_train_entries = (int)(df.shape[0] * train_percentage)
        train = df.head(num_of_train_entries)
        test = df.tail(df.shape[0] - num_of_train_entries)
        
    return train, test

<h4>Functions for Data Format Manipulation</h4>

In [137]:
#
def get_dictionary_from_1darray(keys):
    dictio = {}
    dictio.update({key:0 for key in keys})
    return dictio

#
def get_ndarray_from_dataframe(df):
    validate_instance(df, pd.DataFrame)
    # return df[df.columns[:]].values
    return np.array(df)

#
def extract_desired_features(df, features_list, disregard_capitalization=True):
    validate_instance(df, pd.DataFrame)
    validate_instance(features_list, list)

    if disregard_capitalization:
        features_list: list = [s.lower() for s in features_list if isinstance(s, str)]
    matching_columns: list = [col for col in df.columns if (col.lower() if disregard_capitalization else col) in features_list]
    
    return df[matching_columns]

#
def get_features_and_classification(entry):
    """
    Separates the features of an entry from its classification, assuming the classification is found at the last index.

    :param entry: An entry of an instance that includes both the features and the classification.

    :return: A tuple of the features at index 0 and the classification at index 1.
    """
    features = entry[:-1]
    classification = entry[-1]
    return features, classification

#
def separate_features_and_classifications_for_dataframe(df):
    """
    Separates the features of all the entries in a dataframe from their respective classification.
    Assumes that the classification is found at the last index of every row.

    :param df: A pandas.DataFrame that holds entries with both features and classifications.

    :return: A tuple of the features at index 0 and the classifications at index 1.

    :error: returns -1 if argument df is not an instance of pandas.DataFrame
    """
    validate_instance(df, pd.DataFrame)
    
    features = df[df.columns[:-1]].values
    classifications = df[df.columns[-1]].values
    return features, classifications

#
def separate_features_and_classifications_for_ndarray(arr):
    """
    Separates the features of all the entries in an ndarray from their respective classification.
    Assumes that the classification is found at the last index of every row.

    :param arr: A numpy.ndarray that holds entries with both features and classifications.

    :return: A tuple of the features at index 0 and the classifications at index 1.
    
    :error: returns -1 if argument arr is not an instance of numpy.ndarray
    """
    validate_instance(arr, np.ndarray)
    
    features = arr[:, :-1]
    classifications = arr[:, -1]
    return features, classifications

#
def separate_features_and_classifications_for_df_or_ndarray(table):
    """
    Separates the features of all the entries in a table from their respective classification.
    Assumes that the classification is found at the last index of every row.

    :param table: A numpy.ndarray or a pandas.DataFrame that holds entries with both features and classifications.

    :return: A tuple of the features at index 0 and the classifications at index 1.
    
    :error: returns -1 if argument table is not an instance of either numpy.ndarray or pandas.DataFrame
    """
    if isinstance(table, np.ndarray):
        return separate_features_and_classifications_for_ndarray(table)
    if isinstance(table, pd.DataFrame):
        return separate_features_and_classifications_for_dataframe(table)
    print(f'ERROR!\nArgument must be an instance of either numpy.ndarray or pandas.DataFrame !')
    return -1


<h5>Data Extraction</h5>

In [138]:
df = pd.read_csv('./Resources/Data/data.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [139]:
features_list = ["PassengerId", "HomePlanet", "Destination", "Cabin"]
reduced_df = extract_desired_features(df, features_list, disregard_capitalization=True)
D_train, D_test = select_train_and_test_by_train_percentage(reduced_df, 0.8)

In [140]:
D_train

,PassengerId,HomePlanet,Cabin,Destination
0,0001_01,Europa,B/0/P,TRAPPIST-1e
1,0002_01,Earth,F/0/S,TRAPPIST-1e
2,0003_01,Europa,A/0/S,TRAPPIST-1e
3,0003_02,Europa,A/0/S,TRAPPIST-1e
4,0004_01,Earth,F/1/S,TRAPPIST-1e
...,...,...,...,...
6949,7376_01,Europa,E/477/P,TRAPPIST-1e
6950,7377_01,Earth,G/1186/P,PSO J318.5-22
6951,7379_01,Earth,F/1416/S,TRAPPIST-1e
6952,7385_01,Earth,F/1417/S,PSO J318.5-22


In [141]:
D_test

,PassengerId,HomePlanet,Cabin,Destination
6954,7388_01,Earth,G/1198/S,PSO J318.5-22
6955,7390_01,Earth,G/1190/P,TRAPPIST-1e
6956,7391_01,Earth,G/1191/P,TRAPPIST-1e
6957,7392_01,Earth,G/1192/P,TRAPPIST-1e
6958,7393_01,Earth,E/478/P,NaN
...,...,...,...,...
8688,9276_01,Europa,A/98/P,55 Cancri e
8689,9278_01,Earth,G/1499/S,PSO J318.5-22
8690,9279_01,Earth,G/1500/S,TRAPPIST-1e
8691,9280_01,Europa,E/608/S,55 Cancri e
